# Importation des librairies et données

In [263]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Masking, Embedding, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import nltk

from sklearn import linear_model
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import requests
import json

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [264]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


#Découpage des données

In [265]:
data = pd.read_csv('gdrive/MyDrive/NLP_Projet/data_NLP.csv',encoding='ISO-8859-1',header=None,names=['feeling','ids','date','?','utilisateur','text'])
data.drop(['ids','date','?','utilisateur'], axis=1, inplace=True)  #j'enlève les colonnes que je n'utilise pas
print(data)
data=data[799000:801000]  #on réduis la quantité de tweets (1000 tweets négatifs et 1000 positifs)
print(data)



         feeling                                               text
0              0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1              0  is upset that he can't update his Facebook by ...
2              0  @Kenichan I dived many times for the ball. Man...
3              0    my whole body feels itchy and like its on fire 
4              0  @nationwideclass no, it's not behaving at all....
...          ...                                                ...
1599995        4  Just woke up. Having no school is the best fee...
1599996        4  TheWDB.com - Very cool to hear old Walt interv...
1599997        4  Are you ready for your MoJo Makeover? Ask me f...
1599998        4  Happy 38th Birthday to my boo of alll time!!! ...
1599999        4  happy #charitytuesday @theNSPCC @SparksCharity...

[1600000 rows x 2 columns]
        feeling                                               text
799000        0  CAN'T BEAT LIVE MUSIC, WISH I COULD SING BUT I...
799001        0  Charl

# Utilisation de la classe tokenizer de keras

In [266]:
# La classe tokenizer de keras permet de vectoriser le text corpus.
tokenizer = Tokenizer(num_words=20000, split=" ")
tokenizer.fit_on_texts(data["text"].values)


X= tokenizer.texts_to_sequences(data["text"].values) # convertir les tokens du texte en séquence d'integers
X = pad_sequences(X) 
print(X[:33])
print(X.shape)

[[   0    0    0 ...   21    1  100]
 [   0    0    0 ...  840  841 1750]
 [   0    0    0 ...    7   93  411]
 ...
 [   0    0    0 ...   15   63   22]
 [   0    0    0 ...  456   19 1804]
 [   0    0    0 ...    2 1127   14]]
(2000, 32)


In [267]:

Y=pd.get_dummies(data["feeling"]).values    # renvoie [0 1] pour les positifs et [1 0] pour les négatifs
print(Y)                  



[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]


#RNN Model

In [270]:

model = Sequential()


model.add(Embedding(20000, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256,return_sequences=False,dropout=0.3,recurrent_dropout=0.2))
model.add(Dense(2, activation="sigmoid"))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=0)




Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 32, 256)           5120000   
_________________________________________________________________
dropout_28 (Dropout)         (None, 32, 256)           0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 514       
Total params: 5,645,826
Trainable params: 5,645,826
Non-trainable params: 0
_________________________________________________________________


#Entrainement du modèle

In [271]:
model.fit(x_train,  y_train, epochs=8, batch_size=100,verbose =2)

Epoch 1/8
15/15 - 12s - loss: 0.6747 - accuracy: 0.5960
Epoch 2/8
15/15 - 9s - loss: 0.5053 - accuracy: 0.7767
Epoch 3/8
15/15 - 9s - loss: 0.2962 - accuracy: 0.8753
Epoch 4/8
15/15 - 9s - loss: 0.1623 - accuracy: 0.9400
Epoch 5/8
15/15 - 9s - loss: 0.0771 - accuracy: 0.9747
Epoch 6/8
15/15 - 9s - loss: 0.0460 - accuracy: 0.9920
Epoch 7/8
15/15 - 9s - loss: 0.0716 - accuracy: 0.9780
Epoch 8/8
15/15 - 9s - loss: 0.0505 - accuracy: 0.9940


#Evaluation du modèle

In [272]:
model.evaluate(x_test,y_test)


16/16 [==============================] - 1s 48ms/step - loss: 0.5457 - accuracy: 0.7400


[0.5457485914230347, 0.7400000095367432]

#Sauvegarde du modèle

In [273]:
model.save('/content/gdrive/MyDrive/NLP_Projet/models/Model_1')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/NLP_Projet/models/Model_1/assets


# Récupération des commentaires depuis l'API Yelp

In [274]:
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}
url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
data=json.loads(req.text)
data

{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=KzZVmcfbYqCMY9omoha5NA&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KzZVmcfbYqCMY9omoha5NA',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

#Pipeline permettant de faire une prédiction en mettant une phrase en entrée

In [275]:

    
def pipeline(sentence):

  text_data=pd.DataFrame([sentence], columns = ["text"])
  tokenizer = Tokenizer(num_words=20000, split=" ")
  tokenizer.fit_on_texts(text_data["text"].values)

  texte_traité= tokenizer.texts_to_sequences(text_data["text"].values)
  texte_traité = pad_sequences(texte_traité)

  a=model.predict(texte_traité)

  if a[0][0] < (1/3):
    print('Negatif')
    
  elif  a[0][0] > (2/3): 
    print('Positif')
    
  else:
    print('Neutre')

  return a



#Prediction

In [276]:
review1 =data["reviews"][0]["text"]
review2=data["reviews"][1]["text"]
review3=data["reviews"][2]["text"]

prediction1 = pipeline(review1)
prediction2 = pipeline(review2)
prediction3 = pipeline(review3)


    


Positif
Positif
Positif
